## ДЗ ##
1. Попробуйте починить сеть по словам, сделанную на уроке.

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения.

### Решение ###
В попытках починить сеть и одновременно изменить ее параметры для генерации как можно более осмысленного текста, пришел к выводу о том, что необходимо использование Embedding слоя.
Изменил функцию buildPhrase, увеличил число слов maxWordsCount = 5000 и взял текст побольше. Результаты впечатлили. После 50 эпох accuracy: 0.9960 и первозданный текст Александра Сергеевича.

In [120]:
import re

In [121]:
with open('onegin.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [122]:
text[:100]

'Мой дядя самых честных правилКогда не в шутку занемогОн уважать себя заставилИ лучше выдумать не мог'

In [123]:
len(text)

10663

In [124]:
num_characters = 34 #33 буквы + пробел

In [125]:
import keras
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [126]:
maxWordsCount = 5000
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [127]:
tokenizer.fit_on_texts(text)

In [128]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'а': 4, 'н': 5, 'и': 6, 'т': 7, 'л': 8, 'с': 9, 'в': 10, 'р': 11, 'м': 12, 'д': 13, 'к': 14, 'у': 15, 'п': 16, 'ы': 17, 'ь': 18, 'й': 19, 'г': 20, 'б': 21, 'з': 22, 'я': 23, 'ч': 24, 'ж': 25, 'х': 26, 'ш': 27, 'ю': 28, 'ц': 29, 'ф': 30, 'щ': 31, 'э': 32, 'ъ': 33}


In [81]:
inp_chars = 6 #
data = tokenizer.texts_to_matrix(text)

In [82]:
data[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [83]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

10657

In [84]:
import numpy as np

In [85]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [86]:
len(X)

10657

In [87]:
X[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [88]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [89]:
from keras.layers import *
from keras.models import Sequential

In [90]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 500)               267500    
                                                                 
 dense_3 (Dense)             (None, 34)                17034     
                                                                 
Total params: 284,534
Trainable params: 284,534
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
334/334 [==============================] - 5s 11ms/step - loss: 2.9792 - accuracy: 0.1596
Epoch 2/100
334/334 [==============================] - 3s 10ms/step - loss: 2.7192 - accuracy: 0.2072
Epoch 3/100
334/334 [==============================] - 3s 10ms/step - loss: 2.6403 - accuracy: 0.2199
Epoch 4/100
334/334 [==============================] - 3s 10ms/step - loss: 2.6071 - accuracy: 0.2185
Epoch 5/100
334/334 [==============================] - 4s 11ms/step - loss: 2.5733 - accuracy: 0.2313
Epoch 6/100
334/334 [==============================] - 3s 10ms/step - loss: 2.5686 - accuracy: 0.2221
Epoch 7/100
334/334 [==============================] - 3s 10ms/step - loss: 2.5643 - accuracy: 0.2307
Epoch 8/100
334/334 [==============================] - 3s 10ms/step - loss: 2.5608 - accuracy: 0.2281
Epoch 9/100
334/334 [==============================] - 4s 11ms/step - loss: 2.5466 - accuracy: 0.2322
Epoch 10/100
334/334 [==============================] - 4s 11ms/step - loss: 2.531

334/334 [==============================] - 4s 11ms/step - loss: 1.1900 - accuracy: 0.6255
Epoch 81/100
334/334 [==============================] - 4s 11ms/step - loss: 1.2738 - accuracy: 0.5958
Epoch 82/100
334/334 [==============================] - 3s 10ms/step - loss: 1.1389 - accuracy: 0.6434
Epoch 83/100
334/334 [==============================] - 4s 11ms/step - loss: 1.1581 - accuracy: 0.6356
Epoch 84/100
334/334 [==============================] - 4s 13ms/step - loss: 1.1165 - accuracy: 0.6449
Epoch 85/100
334/334 [==============================] - 4s 13ms/step - loss: 1.0353 - accuracy: 0.6728
Epoch 86/100
334/334 [==============================] - 4s 13ms/step - loss: 1.0328 - accuracy: 0.6725
Epoch 87/100
334/334 [==============================] - 4s 12ms/step - loss: 0.9955 - accuracy: 0.6835
Epoch 88/100
334/334 [==============================] - 4s 13ms/step - loss: 0.9264 - accuracy: 0.7129
Epoch 89/100
334/334 [==============================] - 4s 12ms/step - loss: 0.9247 - 

In [98]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [99]:
res = buildPhrase("Мой дядя")
print(res)

1/1 [==============================] - 0s 146ms/step
Мой дядядя ь меменонеко   ва олс  авб еенсгитах  рннеоом к


# Слова


In [132]:
with open('onegin.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [133]:
maxWordsCount = 5000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [134]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('мой', 10), ('дядя', 1), ('самых', 1), ('честных', 1), ('правил', 1), ('когда', 7), ('не', 26), ('в', 42), ('шутку', 1), ('занемог', 1)]


In [135]:
data = tokenizer.texts_to_sequences([texts])

In [136]:
data

[[15,
  212,
  213,
  214,
  215,
  23,
  4,
  2,
  216,
  217,
  3,
  218,
  58,
  219,
  1,
  89,
  220,
  4,
  16,
  9,
  221,
  222,
  90,
  11,
  91,
  15,
  223,
  224,
  17,
  225,
  226,
  1,
  92,
  1,
  227,
  4,
  228,
  93,
  229,
  230,
  231,
  232,
  233,
  234,
  94,
  38,
  235,
  236,
  237,
  238,
  239,
  240,
  1,
  95,
  241,
  58,
  23,
  31,
  242,
  243,
  244,
  39,
  245,
  246,
  247,
  248,
  2,
  96,
  5,
  249,
  250,
  251,
  252,
  253,
  32,
  59,
  254,
  24,
  97,
  255,
  1,
  256,
  17,
  257,
  258,
  259,
  40,
  260,
  261,
  31,
  262,
  263,
  264,
  41,
  20,
  265,
  15,
  266,
  267,
  5,
  268,
  269,
  25,
  98,
  99,
  270,
  12,
  100,
  271,
  15,
  272,
  13,
  273,
  274,
  1,
  6,
  11,
  275,
  276,
  10,
  277,
  278,
  279,
  2,
  280,
  281,
  282,
  283,
  284,
  285,
  9,
  101,
  286,
  42,
  287,
  288,
  1,
  289,
  102,
  290,
  291,
  292,
  293,
  43,
  60,
  26,
  44,
  294,
  103,
  61,
  295,
  296,
  297,
  21,
  298

In [137]:
len(data[0])

1996

In [138]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )
print( res.shape )

(1996,)


In [139]:
inp_words = 3
n = res.shape[0] - inp_words

In [140]:
X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [141]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(256, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 3, 256)            1280000   
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 5000)              1285000   
                                                                 
Total params: 2,696,328
Trainable params: 2,696,328
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
63/63 [==============================] - 3s 35ms/step - loss: 8.2771 - accuracy: 0.0326
Epoch 2/50
63/63 [==============================] - 2s 35ms/step - loss: 6.9139 - accuracy: 0.0447
Epoch 3/50
63/63 [==============================] - 2s 34ms/step - loss: 6.4656 - accuracy: 0.0462
Epoch 4/50


In [142]:
def buildPhrase(texts, str_len = 40):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res

In [143]:
res = buildPhrase("мой дядя самых")
print(res)

1/1 [==============================] - 0s 165ms/step
мой дядя самых честных правил когда не в шутку занемог он уважать себя заставил и лучше выдумать не мог его пример другим наука но боже мой какая скука с больным сидеть и день и ночь не отходя ни шагу прочь какое низкое коварство


In [112]:
x.shape

(1, 512, 1000)

In [116]:
res = buildPhrase("театр уж полон")
print(res)

1/1 [==============================] - 0s 147ms/step
театр уж полон ложи блещут партер и кресла всё кипит в райке нетерпеливо плещут и взвившись занавес шумит блистательна полувоздушна смычку волшебному послушна толпою нимф окружена стоит истомина она одной ногой касаясь пола другою медленно кружит и вдруг прыжок и вдруг летит летит
